In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import pathlib
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential
from IPython.display import Image
from keras import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from tensorflow.keras import datasets, layers, models, losses
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset_dir = "drive/MyDrive/CIFAKE"
img_height = 32
img_width = 32
batch_size = 500

# Load training and validation data
train, val = keras.utils.image_dataset_from_directory(
  os.path.join(dataset_dir, "train"),
  validation_split=0.2,
  subset="both",
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 98265 files belonging to 2 classes.
Using 78612 files for training.
Using 19653 files for validation.


In [4]:
test = keras.utils.image_dataset_from_directory(
  os.path.join(dataset_dir, "test"),
  seed = 512,
  image_size = (img_height, img_width),
  batch_size = batch_size)

Found 20000 files belonging to 2 classes.


In [5]:
classes = train.class_names
classes

['FAKE', 'REAL']

In [6]:
input = layers.Input(shape=(32,32,3))
input_tensor = layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear", input_shape=(32,32,3))(input)
# 7x7 convolution with stride 2 and 64 filters
x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_tensor)
# 3x3 max pooling with stride 2
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
# 1x1 convolution with 64 filters
x = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
# 3x3 convolution with 192 filters
x = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(x)
# 3x3 max pooling with stride 2
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
#inception 1
branch1 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(96, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(16, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(32, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#inception 2
branch1 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(96, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
#inception 3
branch1 = layers.Conv2D(192, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(96, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(208, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(16, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(48, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#auxiliary 1
auxx = layers.AveragePooling2D((5, 5), strides=(3, 3))(x)
auxx = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(auxx)
auxx = layers.Flatten()(auxx)
auxx = layers.Dense(1024, activation='relu')(auxx)
auxx = layers.Dropout(0.7)(auxx)
auxx = layers.Dense(1, activation='sigmoid', name="aux1")(auxx)
#inception 4
branch1 = layers.Conv2D(160, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(112, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(224, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(24, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(64, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#inception 5
branch1 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(24, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(64, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#inception 6
branch1 = layers.Conv2D(112, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(144, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(288, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(64, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#auxiliary 2
auxx2 = layers.AveragePooling2D((5, 5), strides=(3, 3))(x)
auxx2 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(auxx2)
auxx2 = layers.Flatten()(auxx2)
auxx2 = layers.Dense(1024, activation='relu')(auxx2)
auxx2 = layers.Dropout(0.7)(auxx2)
auxx2 = layers.Dense(1, activation='sigmoid', name="aux2")(auxx2)
#inception 7
branch1 = layers.Conv2D(256, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(160, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(320, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(128, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
#inception 8
branch1 = layers.Conv2D(256, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(160, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(320, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(32, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(128, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)
#inception 9
branch1 = layers.Conv2D(384, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(192, (1, 1), padding='same', activation='relu')(x)
branch2 = layers.Conv2D(384, (3, 3), padding='same', activation='relu')(branch2)
branch3 = layers.Conv2D(48, (1, 1), padding='same', activation='relu')(x)
branch3 = layers.Conv2D(128, (5, 5), padding='same', activation='relu')(branch3)
branch4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
branch4 = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(branch4)
x = layers.concatenate([branch1, branch2, branch3, branch4], axis=-1)

x = layers.AveragePooling2D((7, 7), strides=(1, 1))(x)
x = layers.Dropout(0.4)(x)
x = layers.Flatten()(x)

output = layers.Dense(1, activation='sigmoid', name='output')(x)

model = keras.Model(input, [output, auxx, auxx2], name='inceptionnet')

In [7]:
model.compile(optimizer='adam',
              loss=[losses.binary_crossentropy, losses.binary_crossentropy, losses.binary_crossentropy],
              loss_weights=[1, 0.3, 0.3],
              metrics=["accuracy", metrics.Precision(name="precision"), metrics.Recall(name="recall")])

from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)
model.summary()

Model: "inceptionnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 resizing (Resizing)         (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['resizing[0][0]']            
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 56, 56, 64)           0         ['conv2d[0][0]']              
 D)                                                                                    

In [ ]:
epochs = 40

history = model.fit(
  train,
  validation_data=val,
  epochs=epochs,
  verbose=3, callbacks=[es])

history = pd.DataFrame(history.history)
display(history)

Epoch 1/40


In [ ]:
plt.plot(range(epochs), history["accuracy"], label="Training Accuracy")
plt.plot(range(epochs), history["val_accuracy"], label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training and Validation Accuracy")
plt.show()

In [ ]:
model.evaluate(test, return_dict=True)